In [1]:
pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 48.5 MB/s  0:00:01m0:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.
N

In [3]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import tarfile
import nltk
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

# Download NLTK data
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
# Step 2: Extract dataset files
import tarfile
import os

# Path to the .tgz file in Kaggle environment
cnn_stories_path = "/kaggle/input/nlp-proj/cnn_stories.tgz"
# Directory where you want to extract the files
extraction_path = "/kaggle/working/cnn_stories"  # This is the standard working directory in Kaggle

# Ensure the extraction directory exists
os.makedirs(extraction_path, exist_ok=True)

# Open the .tgz file
with tarfile.open(cnn_stories_path, 'r:gz') as tar:
    # Get all files in the archive
    all_files = tar.getnames()
    
    files_to_extract = all_files
    
    # Extract the selected files
    tar.extractall(path=extraction_path, members=[tar.getmember(file) for file in files_to_extract])

print(f"Extracted the files to {extraction_path}")

Extracted the files to /kaggle/working/cnn_stories


In [6]:
# Step 3: Define the story parser
def parse_story_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
        parts = content.split('@highlight')
        article = parts[0].strip()
        highlights = [h.strip() for h in parts[1:] if h.strip()]
        highlights = ' '.join(highlights)
        return {'article': article, 'highlights': highlights}

In [7]:
# Step 4: Process all CNN stories
def process_stories(story_dir):
    data = []
    for root, _, files in os.walk(story_dir):
        for file in files:
            if file.endswith('.story'):
                try:
                    data.append(parse_story_file(os.path.join(root, file)))
                except Exception as e:
                    print(f"Error processing {file}: {e}")
    return data

print("Processing CNN stories...")
cnn_data = process_stories('cnn_stories/cnn/stories')

Processing CNN stories...


In [8]:
# Step 5: Create train/val/test splits
train_data, temp_data = train_test_split(cnn_data, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

In [9]:
# Step 6: Create HuggingFace Dataset
dataset = DatasetDict({
    'train': Dataset.from_list(train_data),
    'validation': Dataset.from_list(val_data),
    'test': Dataset.from_list(test_data)
})

In [10]:
# Step 7: Initialize tokenizer and preprocess
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_function(batch):
    inputs = tokenizer(
        batch["article"],
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    outputs = tokenizer(
        batch["highlights"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["labels"] = outputs.input_ids

    # Replace padding token id with -100 for loss calculation
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
# Step 8: Apply preprocessing
encoded_dataset = dataset.map(
    preprocess_function,
    batched=True,
    batch_size=32,
    remove_columns=["article", "highlights"]
)

Map:   0%|          | 0/74063 [00:00<?, ? examples/s]

Map:   0%|          | 0/9258 [00:00<?, ? examples/s]

Map:   0%|          | 0/9258 [00:00<?, ? examples/s]

In [ ]:
# Step 9: Set format for PyTorch
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [ ]:
# Step 10: Verify the processed dataset
print("Processed dataset structure:", encoded_dataset)
print("Sample input IDs:", encoded_dataset["train"][0]["input_ids"])
print("Sample attention mask:", encoded_dataset["train"][0]["attention_mask"])
print("Sample labels:", encoded_dataset["train"][0]["labels"])

In [ ]:
# Save processed dataset
encoded_dataset.save_to_disk("/kaggle/working/processed_cnn_stories")